## Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['GoogleNews-vectors-negative300.bin.gz', 'COPY!!!!!!!!!!!!!!!!!.ipynb', 'gpt2-gerchef', 'Copy of Fine-tune a non-English GPT-2 Model with Huggingface', 'tess_copy_commentgeneration.ipynb', 'LifeProTips', 'UIUC', 'bigtenLinearSVC.pickle', 'model1.ipynb', 'UIUC2', 'preprocessCornell.ipynb', 'uofm', 'askredditpreprocessing.ipynb', 'gpt-2-with-huggingface-pytorch.ipynb', 'Test_Evaluation_Metrics.ipynb.ipynb', 'commentgeneration.ipynb', 'train_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'test_com_set.pickle', 'todayilearnedgeneration.ipynb', 'val_post_set.pickle', 'val_com_set.pickle', 'COPY FOR TRYING SEQTOSEQ uiucgenerationmanual.ipynb', 'Cornell Final Train.ipynb', 'Illinois', 'UMD', 'Indiana', 'Iowa', 'msu', 'uofmn', 'Indiana Final Train.ipynb', 'Penn', 'Ohio', 'Penn Final Train.ipynb', 'uofmn Final Train.ipynb', 'lincoln', 'Northwestern', 'rutgers', 'UWMadison', 'Cornell (OLD FOLDER)', 'Cornell', 'Purdue', 'Iowa Train Final.ipynb', 'Ohio

In [4]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.1 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=2

In [8]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [10]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA = "448 Group: Jedi Order/data/CornellSubreddits/UIUC.corpus"
DATA_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_DATA)
# corpus = Corpus(filename=download("subreddit-Cornell"))
corpus = Corpus(filename=DATA_PATH)
corpus.print_summary_stats()

Number of Speakers: 35751
Number of Utterances: 657989
Number of Conversations: 66177


In [ ]:
convo_df = corpus.get_conversations_dataframe()
utt_df = corpus.get_utterances_dataframe()
utt_ids = corpus.get_utterance_ids()

In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)
temp.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,vectors
id,,,,,,,,,,,,,,,
c9mqrap,1366935211,"Just sent:\n\nDear Mr. Long,\n\nI'm an attorne...",KenPopehat,1d3qqc,1d3qqc,1893,c9mqrap,1431426942,3,None,UIUC,False,,.,[]
1d3qqc,1366919148,\n**Update: [no lawsuit](http://www.reddit.com...,NegatedVoid,None,1d3qqc,812,None,1412707227,0,None,UIUC,False,/r/UIUC/comments/1d3qqc/my_correspondence_with...,In California where it is warm! :P,[]
7xv7vd,1518743608,,allen980123,None,7xv7vd,765,None,1520544467,0,None,UIUC,False,/r/UIUC/comments/7xv7vd/thank_you_for_making_t...,,[]
e8sxfpk,1541010532,It was a toy grenade.\n\nYou weren't expecting...,UIpolice,9t1b2e,9t1b2e,742,e8sxfpk,1541677641,0,"{'gid_1': 0, 'gid_2': 0, 'gid_3': 0}",UIUC,False,/r/UIUC/comments/9t1b2e/pls_respond_i_need_to_...,University of Illinois Police Department,[]
7gfnju,1511981772,,Peoria_Charter_Coach,None,7gfnju,737,None,1512323902,0,None,UIUC,False,/r/UIUC/comments/7gfnju/suburban_express_hypoc...,Peoria Charter Coach Company,[]


In [ ]:
temp = utt_df.sort_values("meta.score", ascending=False)

posts = temp[temp['reply_to'].isnull()]
comments = temp[temp['reply_to'].notnull()]

posts = posts[(posts["text"] != '') & (posts["text"] != '[deleted]')  & (posts["text"] != '[removed]') & (posts["text"] != ' ')]
comments = comments[(comments["text"] != '') & (comments["text"] != '[deleted]')  & (comments["text"] != '[removed]') & (comments["text"] != ' ')]

conversation_ids_in = list(posts["conversation_id"])

coversation_ids = []
num_ids = 0
for id in conversation_ids_in:
  if num_ids == 20000:
      break
  dataframe = comments[comments["conversation_id"] == id]
  com_ids = list(dataframe.index)
  if com_ids != []:
    coversation_ids.append(id)
    num_ids = num_ids + 1

top_posts = posts.head(20000)
top_posts.head()

print(len(top_posts))


1d3qqc
20000


In [ ]:
coversation_ids = coversation_ids[:20000] #not sure if needed 

In [12]:
datasets_to_use = []
train_dataset_to_use = []
val_dataset_to_use = []
test_dataset_to_use = []
train_com = []
val_com = []
test_com = []
train_len = []
val_len = []
test_len = []
train_com_len = []
val_com_len = []
test_com_len = []

In [ ]:
import re
from nltk import word_tokenize
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split

tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
train_ids, test_ids = train_test_split(coversation_ids, test_size=0.25, random_state=42)
val_ids, test_ids = train_test_split(test_ids, test_size=0.5, random_state=42)
ids_total = [train_ids, val_ids, test_ids]
post_len = [train_len, val_len, test_len]
com_len = [train_com_len, val_com_len, test_com_len]
posts_datasets_to_use = [train_dataset_to_use, val_dataset_to_use, test_dataset_to_use]
comments_datasets_to_use = [train_com, val_com, test_com]
for index, id_list in enumerate(ids_total):
  for id in id_list:
      dataframe = comments[comments["conversation_id"] == id]
      post = posts.loc[id]['text']
      post = re.sub("\n", " ", post)
      post = re.sub("&gt;", "", post)
      post = re.sub("  ", " ", post)
      post = re.sub("&amp;", "", post)
      post = re.sub("#x200B;", "", post)
      post = re.sub(r'http\S+', '[url link]', post)
      post_tokens = word_tokenize(post)
      len_post = len(post_tokens)
      post_len[index].append(len_post)
      commented_here = ""
      com_ids = list(dataframe.index)
      if (len_post <= 512):
          running_sum = 0 # testing
          for id_c in com_ids:
            out_text = dataframe.loc[id_c]['text']
            out_text = str(out_text).strip()
            out_text = re.sub(r"\s", " ", out_text)
            out_text = re.sub(r"\n", " ", out_text)
            out_text = re.sub("&gt;", "", out_text) # added preprocess
            out_text = re.sub("  ", " ", out_text) # added preprocess
            out_text = re.sub("&amp;", "", out_text) # added preprocess
            out_text = re.sub("#x200B;", "", out_text) # added preprocess
            out_text = re.sub(r'http\S+', '[url link]', out_text) # added preprocess
            out_text_tokens = word_tokenize(out_text) # added this
            running_sum += len(out_text_tokens) # added this
            commented_here = commented_here + " " + out_text
            if running_sum > len_post: # changed this to 512
              break # added this
          comment_tokens = word_tokenize(commented_here)
          com_len[index].append(len(comment_tokens))
          post_utter = "<bos> " + post + " <eos> "
          comment_utter = "<bos>" + commented_here + " <eos> "
          posts_datasets_to_use[index].append(post_utter)
          comments_datasets_to_use[index].append(comment_utter)
print(posts_datasets_to_use[0][:10])
print(comments_datasets_to_use[0][:10])
print(len(posts_datasets_to_use[0]))
print(len(comments_datasets_to_use[0]))
print(post_len[0][0])
print(com_len[0][0])
print(len(post_len[0]))
print(len(com_len[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


["<bos> Hi there,  I live around Green and 1st (near the Maize Mexican grill restaurant). I'll like to know if there's any particularly good places to run such as parks, trails, etc. I run about 4 miles. Also, any neighborhoods that I should avoid for safety reasons? Many thanks! <eos> ", "<bos> I've tried reading up on all of this stuff, but it just seems too wordy. I get what the issue is; now I want to know what happens if TA's stop working. What classes/departments are going to be hit hardest? What happens in undergrad classes that rely on TAs for grades? What happens to those grades? What happens to the rest of the semester? Help!  <eos> ", "<bos> Hi there,  I live around Green and 1st (near the Maize Mexican grill restaurant). I'll like to know if there's any particularly good places to run such as parks, trails, etc. I run about 4 miles. Also, any neighborhoods that I should avoid for safety reasons? Many thanks! <eos> ", "<bos> I've tried reading up on all of this stuff, but it

In [ ]:
averages_post = []
boundary_post = []
averages_com = []
boundary_com = []
train_post_remove = []
val_post_remove = []
test_post_remove = []
posts_removed_ind = [train_post_remove, val_post_remove, test_post_remove]
train_com_remove = []
val_com_remove = []
test_com_remove = []
coms_removed_ind = [train_com_remove, val_com_remove, test_com_remove]

for datasets in post_len:
  temp = np.array(datasets)
  averages_post.append(np.mean(temp))
  boundary_post.append(np.percentile(temp, 96))
for datasets in com_len:
  temp = np.array(datasets)
  averages_com.append(np.mean(temp))
  boundary_com.append(np.percentile(temp, 75))

print(averages_post)
print(boundary_post)
print(averages_com)
print(boundary_com)

[88.16744278871741, 88.8876, 87.2208]
[301.0, 313.03999999999996, 295.03999999999996]
[114.40157798907546, 114.5156059991893, 114.12479674796748]
[143.0, 145.0, 143.25]


In [ ]:
train_tensors = []
val_tensors = []
test_tensors = []
train_com_tensors = []
val_com_tensors = []
test_com_tensors = []
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(len(tokenizer))

50260


In [ ]:
post_tensors_data = [train_tensors, val_tensors, test_tensors]
comments_tensors_data = [train_com_tensors, val_com_tensors, test_com_tensors]
# there doesn't seem to be a way to associate the two permanently, since tokenizer automatically removes it.
# maybe the exception is you reach into the other comment and remove?
# for instance, loop with like index: if it excepts, then: you go into the other coms dataset_to_use and remove that specific index.
# say in train, which is index = 0, and we find that at index1, post is too long. go into coms_dataset_to_use and delete that entry.
# i assume that it does the same thing for the other one. 
post_max = []
com_max = []
for index, dataset_tensors in enumerate(posts_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    post_tensors_data[index].append(text)
    #except:
    #print(index1) # watch for big tokens how to. like does this already truncate? 
    #del comments_datasets_to_use[index][index1]
    #del posts_datasets_to_use[index][index1] #maybe this one too for the next pass around
      # only problem is are the indexes reliable? like does index1 mean the same thing in both when you delete
      # and does a failed tokenizer actually do anything

for index, dataset_tensors in enumerate(comments_datasets_to_use):
  for index1, text in enumerate(dataset_tensors):
    comments_tensors_data[index].append(text)

#for index, dataset_tensors in enumerate(posts_datasets_to_use):
#  for index1, text in enumerate(dataset_tensors):
#    encoded_input = tokenizer(text, return_tensors="pt", padding=True)
#    tensors_data[index].append(encoded_input)

In [ ]:
print(post_tensors_data[0][0])

<bos> Hi there,  I live around Green and 1st (near the Maize Mexican grill restaurant). I'll like to know if there's any particularly good places to run such as parks, trails, etc. I run about 4 miles. Also, any neighborhoods that I should avoid for safety reasons? Many thanks! <eos> 


In [15]:

GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO = "448 Group: Jedi Order/models/Illinois"
PATH_TO_PUT = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE_INFO)
print(os.listdir(PATH_TO_PUT))

448 Group: Jedi Order/models/Illinois
['train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'checkpoints-illinois']


In [16]:
train_post_path = os.path.join(PATH_TO_PUT, "train_post_set.pickle")
val_post_path = os.path.join(PATH_TO_PUT, "val_post_set.pickle")
test_post_path = os.path.join(PATH_TO_PUT, "test_post_set.pickle")
train_com_path = os.path.join(PATH_TO_PUT, "train_com_set.pickle")
val_com_path = os.path.join(PATH_TO_PUT, "val_com_set.pickle")
test_com_path = os.path.join(PATH_TO_PUT, "test_com_set.pickle")

In [ ]:


pickle.dump(posts_datasets_to_use[0], open(train_post_path, "wb"))
pickle.dump(posts_datasets_to_use[1], open(val_post_path, "wb"))
pickle.dump(posts_datasets_to_use[2], open(test_post_path, "wb"))
pickle.dump(comments_datasets_to_use[0], open(train_com_path, "wb"))
pickle.dump(comments_datasets_to_use[1], open(val_com_path, "wb"))
pickle.dump(comments_datasets_to_use[2], open(test_com_path, "wb"))

drive/My Drive/448 Group: Jedi Order/models/Illinois/train_post_set.pickle


In [18]:
posts_datasets_to_use = [[],[],[]]
comments_datasets_to_use = [[],[],[]]

In [19]:
posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
posts_datasets_to_use[1] = pickle.load(open(val_post_path, "rb"))
posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
comments_datasets_to_use[1] = pickle.load(open(val_com_path, "rb"))
comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
import tensorflow
### dataset.py ###
class RedditDataset(Dataset):
  """Dataset for our Reddit thing"""

  def __init__(self, posts, comments, dataset_type: str = 'Train'):
    super().__init__()
    self.posts = posts
    self.comments = comments
    self.dataset_type = dataset_type
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.posts)

  def get_labels(self) -> pd.Series:
    return self.comments
  
  def __getitem__(self, idx):
    post_encoding = self.posts[idx]
    comment_encoding = self.comments[idx]
    if self.dataset_type == 'Train':
      token = self.tokenizer.encode(post_encoding + 'TL;DR' + comment_encoding, return_tensors="pt", 
                                    padding="max_length", truncation=True)
    return token

In [ ]:
print(post_tensors_data[0][0])
print(comments_tensors_data[0][0])

<bos> Hi there,  I live around Green and 1st (near the Maize Mexican grill restaurant). I'll like to know if there's any particularly good places to run such as parks, trails, etc. I run about 4 miles. Also, any neighborhoods that I should avoid for safety reasons? Many thanks! <eos> 
<bos> This is how I see the town. You want to avoid certain parts of east Urbana, avoid going too far north, and I would hesitate about running on campus. You're pretty much perfectly safe if you go southwest toward the country club or due south along Neil toward Devonshire/Windsor/Curtis.  [url link] As a townie, I 100% agree with your map <eos> 


In [ ]:
#print(len(tensors_data[index][0]["input_ids"][0]))
#print(len(tensors_data[index][0]["attention_mask"][0]))
#print(encoded_input["input_ids"].shape[1])
#print(encoded_input["attention_mask"].shape[1])

#train_to = zip(post_tensors_data[0], comments_tensors_data[0])
train_data = RedditDataset(post_tensors_data[0], comments_tensors_data[0])
#val_to = zip(post_tensors_data[1], comments_tensors_data[1])
val_data = RedditDataset(post_tensors_data[1], comments_tensors_data[1])
#test_to = zip(post_tensors_data[2], comments_tensors_data[2])
test_data = RedditDataset(post_tensors_data[2], comments_tensors_data[2])
#print(len(comments_tensors_data[0]))
#print(len(posts_datasets_to_use[0]))
#print(len(comments_datasets_to_use[0]))

In [ ]:
# model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
#from transformers import TextDataset
#print(len(train_dataset_to_use))
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
#OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Cornell/checkpoints")
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Illinois/checkpoints-illinois")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    # do_train=True,
    num_train_epochs=1, # change this back to 2 
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy = "steps", #comment this
    eval_steps = 200,
    save_steps= 800,
    warmup_steps = 200,
    load_best_model_at_end = True, #comment this
    prediction_loss_only = True,
    )
# try and fix the above params as well
# convert dataset_to_use from a dictionary to a dataset type so that we can use it in the trainer
train_dataset_to_use_converted = torch.utils.data.DataLoader(train_dataset_to_use)
#print(type(train_dataset_to_use_converted))
test_dataset_to_use_converted = torch.utils.data.DataLoader(test_dataset_to_use)

#early_stop = EarlyStoppingCallback(5, 1.0) #comment this

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_data,
    eval_dataset=test_data
)

# depends on whether we want to use Trainer or not. I think using Trainer would be better if we get it to work,
# but I'm not sure how feasible it is. Additionally manual is kinda what we did in class
# ask about batching our data, and ask about num_labels. Batching our data in that dataloader part. like how to iterate over batches of data.
# idk if need to batch

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
200,No log,3.973226
400,No log,3.613288
600,10.803000,3.543694
800,10.803000,3.523229
1000,3.623700,3.497794
1200,3.623700,3.489264
1400,3.623700,3.470153
1600,3.601500,3.461798
1800,3.601500,3.444937
2000,3.564900,3.438979


TrainOutput(global_step=14829, training_loss=3.6822196790152533, metrics={'train_runtime': 24931.4985, 'train_samples_per_second': 0.595, 'train_steps_per_second': 0.595, 'total_flos': 7749399085056000.0, 'train_loss': 3.6822196790152533, 'epoch': 1.0})

In [ ]:
trainer.save_model()

VVVVVVVVrun if needed

In [ ]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, "Illinois/checkpoints-illinois")
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

^^^^^^run if needed 

In [ ]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
post_in_one = "I saw a squirrel today. I feel blessed. So fluffy. I saw it eat cute nuts."
post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_with_bos = "<bos>" + post_in + " TL;DR"
encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
    )
generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))
# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [ ]:
def make_sent(input):
    output = ""
    end_things = ".!?"
    last_end = 0
    i = 0
    while i < len(input):
        if input[i].isascii() or "'":
            if input[i] in end_things:
                output = output + input[i] + "\n"
                i = i + 1
                last_end = i
                if i + 1 < len(input):
                    if input[i] == " ":
                        i = i + 1
                        if i < len(input):
                            while input[i] == " ":
                                i = i + 1
                                if i >= len(input):
                                    break
                            output = output + " "  
            else:
                if i + 1 == len(input):
                    if input[i] == ",":
                        output = output + "."
                    else:
                        output = output + input[i]
                        output = output + "."
                else:
                    output = output + input[i]
                i = i + 1
    return output

In [ ]:
#https://python.plainenglish.io/use-python-to-split-sentences-into-separate-lines-17a577f9ddb3
print("Post: ")

post_out_now = make_sent(post_in)
print(post_out_now)

i = 1
for x in generated_sequences:
    out = x.split("TL;DR")
    comment = out[1]
    print("Comment {}: ".format(i))
    comment_out_now = make_sent(comment)
    print(comment_out_now)
    i = i+1
    print('\n')

Post: 
I want to go to hotel management school for rural bed and breakfasts.
 What experiences or classes should I seek out?

Comment 1: 
 I studied at the University of Wisconsin for a year and one of my professors said that in one year it might be the easiest class I can get into, which is a shame because this is only a semester old.
 I'm a freshman next year and am a single man who loves to eat and have fun.
 I had good experiences with an alum at.


Comment 2: 
 You can find more info here: [url link]    .


Comment 3: 
 [url link] I'm down to the local bars and restaurants to get my hands on their hotel room.
 Also, if you go into a hotel room next to the Illini Union (where I go for hotel rooms), they give you a key for you to lock it on the bathroom keypad.
 You can go in and lock it on.
 If they lock.




Validation

In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=be1d80e81eeeb3a64c3cffca528e610d00894c928fc26051d01da5393f123f54
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  post_with_bos = "<bos>" + post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(comments_datasets_to_use[1])
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(comments_datasets_to_use[1]):
  val_dataset[cnt]['reference_summary'] = item

for cnt, post in enumerate(post_tensors_data[1]):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: Does anybody have any experience with the French Proficiency Test. I am trying to get out of 3rd level language requirements. How would one go about studying for it?    Just read a bunch of advice on the website. You need to try and get out of the math classes by yourself if you aren't into that. Some of them require an A in the class, if you don't want to take them at the same time (that is in the French department), and it usually requires an F to be in the final. I've
Reference COMMENTS:<bos> I took it my freshman year. I took French all through high school, and before that I'd already taken it for a year in middle school, so it was still pretty fresh in my mind when I took it, but here's what I did. I just looked over all of my notes and tests from the highest two levels, and looked around on the internet for a few practice tests. Review your basic grammar stuff. Make sure you've really got it all down. It probably wouldn't hurt to go over vocab lists from your 

In [ ]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

ValueError: ignored

Sentiment

In [ ]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 13.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [ ]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  print(sent_gt)
  print(gt_label)
  print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  print(sent_generated_comment)
  print(generated_comment_label)
  if idx == 15:
    break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

Reference comments: <bos> 3 hours.. we'll sorta cheat here, and just draw a circle with a radius from Champaign to Chicago. Ok? Interstate speeds differ, and whatever.  Chicago, Indianapolis, Quad Cities, and maybe St Louis.  what's within there? Several State and County Parks. Over in Indiana you have Turkey Run, and up North in Utica you have Starved Rock.  Indianapolis has their Museum of Art, which is pretty nice. St Louis has the City Museum, which is fucking awesome. The Quad Cities.. meh, not sure exactly you're in to. Peoria can have some nice stuff. <eos> 
{'neg': 0.089, 'neu': 0.753, 'pos': 0.158, 'compound': 0.8118}
neutral
Generated comments: Hey guys, I am wondering what are the good places to visit within about 3 hours of driving. I have been to Lake of the Woods as well as Allerton Park before and I like them very much. Are there any other good parks or nice towns to visit? Thanks in advance!     North of Green and Prospect is great in terms of walking. The Park District

In [ ]:
for item in comments_datasets_to_use[1]:
  print(item)